In [1]:
import sys
sys.path.append("src")

import time
from time import sleep

# Iniciem el temporitzador
start_time = time.time()

from driver import setup_driver
######## 1. Driver
driver = setup_driver()

from login import login
from config import USER_KEY, PASS_KEY
######## 2. Login KINEO
url = 'https://aforadores.mitma.es/contadorestraficofomento/Login.aspx'
username = USER_KEY
password = PASS_KEY
login(driver, url, username, password)

######## 3. Pàgina d'aforamens agrupats 1h
driver.get('https://aforadores.mitma.es/contadorestraficofomento/InformePorHorasCalzadaCarrilAforo.aspx')

from dropdown import select_dropdown_value, get_dropdown_values

######## 4. Desplegable 'Demarcación'
input_id_dem = "ctl00_ContentPlaceHolderDatos_CbDemarcacion_I"
dropdown_button_id_dem = "ctl00_ContentPlaceHolderDatos_CbDemarcacion_B-1"
value_to_select_dem = "AP7 BARCELONA SECTOR 4" 
select_dropdown_value(driver, input_id_dem, dropdown_button_id_dem, value_to_select_dem)

sleep(1)

######## 5. Llegir valors desplegable 'ETD'
input_id_etd = "ctl00_ContentPlaceHolderDatos_CbEtd_I"
dropdown_button_id_etd = "ctl00_ContentPlaceHolderDatos_CbEtd_B-1"
# dropdown_values_etd = get_dropdown_values(driver, dropdown_button_id_etd) 

dropdown_values_etd = ['AP7-125+650-100270000000', 'AP7-129+500-100280000000', 'AP7-134+050-100290000000']

print("opcions Desplegable 'ETD' llegides:")
print(f"{dropdown_values_etd}\n")

sleep(1)

from button import click_button, download_excel

from dates import select_date

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

from utils import print_elapsed_time, check_no_data_message

######## 6. Iterar a través de tots els valors del desplegable 'ETD'
for value_to_select_etd in dropdown_values_etd:

    # Desplegable 'ETD'
    select_dropdown_value(driver, input_id_etd, dropdown_button_id_etd, value_to_select_etd)

    sleep(1)

    ######## 7. Desplegable 'Desglose' 
    # input_id_des = "ctl00_ContentPlaceHolderDatos_CbDesgloseMinutos_I"
    # dropdown_button_id_des = "ctl00_ContentPlaceHolderDatos_CbDesgloseMinutos_B-1"
    # value_to_select_des = "15"
    # select_dropdown_value(driver, input_id_des, dropdown_button_id_des, value_to_select_des)
    
    # sleep(1)
    
    ######## 8. Selector de dates
    input_id_ini = 'LbFechaInicio_I'
    value_to_select_ini = '01/06/2024 00:00:00'
    select_date(driver, input_id_ini, value_to_select_ini)
    
    input_id_fin = 'LbFechaFin_I'
    value_to_select_fin = '02/06/2024 00:00:00'
    select_date(driver, input_id_fin, value_to_select_fin)
    
    sleep(1)

    # FALTA SABER SI VOL DESGLOSE PER CALZADA O CARRIL

    ######## 9. Clicar el botó 'Ver' per a veure el llistat
    button_id_ver = "ctl00_ContentPlaceHolderDatos_BtVerListado_I"
    click_button(driver, button_id_ver)

    # Esperar per a deixar que aparegui el botó de "LoadingPanel"
    sleep(2)

    # Esperar a què es carregui la taula
    while True:
        try:
            # Comprova si el "LoadingPanel" és visible
            WebDriverWait(driver, 1).until( 
                EC.visibility_of_element_located((By.ID, "LoadingPanel"))
            )
        except TimeoutException:
            break

    sleep(2)

    # Mentre no aparegui el missatge de "No hay datos para mostrar", es baixarà l'Excel
    if not check_no_data_message(driver):
    
        ######## 10. Clicar el botó per descarregar l'Excel
        button_id_exc = "ctl00_ContentPlaceHolderDatos_BtExcel_I"
        download_excel(driver, button_id_exc) 
        
        sleep(15)  # Esperar que es completi la descàrrega 
        
    else:
        print(f"La ETD {value_to_select_etd} no conté dades a la seva taula. No es baixarà cap document Excel.")

    # printar temps transcorregut
    print_elapsed_time(start_time)
    print()

driver.quit()

Valor seleccionat: AP7 BARCELONA SECTOR 4

opcions Desplegable 'ETD' llegides:
['AP7-125+650-100270000000', 'AP7-129+500-100280000000', 'AP7-134+050-100290000000']

Valor seleccionat: AP7-125+650-100270000000

Temps transcorregut: 0:00:48.795295

Valor seleccionat: AP7-129+500-100280000000

Temps transcorregut: 0:01:17.660595

Valor seleccionat: AP7-134+050-100290000000

Temps transcorregut: 0:01:48.303442

